In [1]:
# creates a porous flow abacus model in a block

from numpy import *
import os

# change from here
# fellmy FluidFlow RHS AA'

horizSize=9500.0
XLeftSlope=4000.0 #begin of the slope
XRightSlope=9500.0 #end of the slope

#daniel's horizNodes=720
horizNodes=200
vertSizeMin=4000.0
vertSizeMax=5000.0
vertNodes=160
#daniel's vertNodes=360

#XHP1=1000.0 #begin left fault position
#XHP2=1100.0 #end left fault position
XHP3=4000.0  #begin right fault position
XHP4=4500.0  #end right fault position
XHP5=6500.0   #left boundary to Recharge Zone
YHP1=4000.0   #top shallow aquifer
YHP2=3000.0   #bottom shallow aquifer

rechargePorosity= 0.167 #Recharge Zone #0.0056-0.167D
RHSPorosityDry= 0.08 #Dry Reservoir #0.006-0.08D
RHSPorosity= 0.167 #RHS Reservoir #0.0056-0.167D
faultPorosity= 0.5 #Opal Mound Fault
shallowAquiferPorosity= 1.56 #Shallow Aquifer

## No right fault in this orientation, only left fault
## change until here 

incX=1
incY=horizNodes
elementType='CPE4P'
gravity=10.

nodeNumber=array(zeros((horizNodes,vertNodes),int))
elemNumber=array(zeros((horizNodes-1,vertNodes-1),int))
coordsX=array(zeros((horizNodes,vertNodes),float))
coordsY=array(zeros((horizNodes,vertNodes),float))
field1=array(zeros((horizNodes,vertNodes),float))
field2=array(zeros((horizNodes,vertNodes),float))
connectivity=array(zeros((horizNodes*vertNodes,4),int))

correctedVertSize=array(zeros(horizNodes,float))

#create the geometry 
def createGeometryFile(geometryFileName):
        g = open(geometryFileName, 'w')
        g.write('** geometry with '+str(horizNodes)+' horizontal nodes and '+str(vertNodes)+' vertical nodes'+'\n')
        g.write('*NODE,NSET=NODES'+'\n')

        for iy in range(vertNodes):
            for ix in range(horizNodes):
                nodeNumber[ix,iy]=(ix+1)*incX+(iy)*incY
                coordsX[ix,iy]=ix*horizSize/horizNodes	

        for ix in range(horizNodes):    
            if coordsX[ix,0]>XRightSlope:
                correctedVertSize[ix]=vertSizeMax
            elif coordsX[ix,0]<XLeftSlope:
                correctedVertSize[ix]=vertSizeMin
            else:
                correctedVertSize[ix]=vertSizeMin+(vertSizeMax-vertSizeMin)*(coordsX[ix,0]-XLeftSlope)/(XRightSlope-XLeftSlope)
		#correctedVertSize[ix]=vertSizeMin+(vertSizeMax-vertSizeMin)*(horizSize-coordsX[ix,0])/(horizSize-peakPosition)

        for iy in range(vertNodes):
        	for ix in range(horizNodes):
                    coordsY[ix,iy]=iy*correctedVertSize[ix]/vertNodes
                    g.write(str(nodeNumber[ix,iy])+',' +str(coordsX[ix,iy])+','+str(coordsY[ix,iy])+'\n')

        g.write('*ELEMENT,TYPE='+elementType+',ELSET=BOX'+'\n')
        for iy in range(vertNodes-1):
            for ix in range(horizNodes-1):
                elemNumber[ix,iy]=(ix+1)*incX+(iy)*incY
                node1=elemNumber[ix,iy]
                node2=elemNumber[ix,iy]+incX
                node3=elemNumber[ix,iy]+incX+incY
                node4=elemNumber[ix,iy]+incY
                connectivity[elemNumber[ix,iy]]= [node1,node2,node3,node4]
                g.write(str(elemNumber[ix,iy])+',' +str(node1)+','+str(node2)+','+str(node3)+','+str(node4)+'\n')
			

        g.write('*NSET,NSET=BOTTOM'+'\n')
        for ix in range(horizNodes):
            g.write(str(nodeNumber[ix,0])+'\n')
            g.write('*NSET,NSET=TOP'+'\n')
        for ix in range(horizNodes):
            g.write(str(nodeNumber[ix,vertNodes-1])+'\n')
            g.write('*NSET,NSET=LEFT'+'\n')
        for iy in range(vertNodes):
            g.write(str(nodeNumber[0,iy])+'\n')
            g.write('*NSET,NSET=RIGHT'+'\n')
        for iy in range(vertNodes):
            g.write(str(nodeNumber[horizNodes-1,iy])+'\n')

        #g.write('*NSET,NSET=LOWPERMEABILITY\n')
        #for iy in range(vertNodes):
        #    for ix in range(horizNodes):
        #        if (coordsX[ix,iy]>=XHP5) or ( (coordsX[ix,iy]>XHP2) and (coordsX[ix,iy]<=XHP5) ) or ( (coordsX[ix,iy]<XHP1) and (coordsY[ix,iy]<YHP2) ):
        #            g.write(str(nodeNumber[ix,iy])+'\n')
        #g.write('*NSET,NSET=HIGHPERMEABILITY\n')
        #for iy in range(vertNodes):
        #    for ix in range(horizNodes):
        #        if ( (coordsX[ix,iy]>=XHP1) and (coordsX[ix,iy]<=XHP2) ) or ( (coordsX[ix,iy]<=XHP1) and (coordsY[ix,iy]>YHP2) and (coordsY[ix,iy]<=YHP1) ):
        #            g.write(str(nodeNumber[ix,iy])+'\n')        	
        
        g.write('*NSET,NSET=RECHARGEDOMAIN\n')
        for iy in range(vertNodes):
            for ix in range(horizNodes):
                if (coordsX[ix,iy]>=XHP5):
                    g.write(str(nodeNumber[ix,iy])+'\n')
        g.write('*NSET,NSET=SHALLOWAQUIFER\n')
        for iy in range(vertNodes):
            for ix in range(horizNodes):
                if ( (coordsY[ix,iy]>=YHP2) and (coordsX[ix,iy]<XHP3) ):
                    g.write(str(nodeNumber[ix,iy])+'\n')
        g.write('*NSET,NSET=FAULTS\n')
        for iy in range(vertNodes):
            for ix in range(horizNodes):
                if (coordsX[ix,iy]>=XHP3) and (coordsX[ix,iy]<=XHP4) : 
                    g.write(str(nodeNumber[ix,iy])+'\n')        	
        g.write('*NSET,NSET=RHS\n')
        for iy in range(vertNodes):
            for ix in range(horizNodes):
                if ( (coordsX[ix,iy]>XHP4) and (coordsX[ix,iy]<XHP5) ):
                    g.write(str(nodeNumber[ix,iy])+'\n')
        g.write('*NSET,NSET=RHSDRY\n')
        for iy in range(vertNodes):
            for ix in range(horizNodes):
                if ( (coordsY[ix,iy]<YHP2) and (coordsX[ix,iy]<XHP3) ):
                    g.write(str(nodeNumber[ix,iy])+'\n')

        g.write('*ELSET,ELSET=BOTTOM'+'\n')
        for ix in range(horizNodes-1):
            g.write(str(nodeNumber[ix,0])+'\n')
        g.write('*ELSET,ELSET=TOP'+'\n')
        for ix in range(horizNodes-1):
            g.write(str(nodeNumber[ix,vertNodes-2])+'\n')
        g.write('*ELSET,ELSET=LEFT'+'\n')
        for iy in range(vertNodes-1):
            g.write(str(nodeNumber[0,iy])+'\n')
        g.write('*ELSET,ELSET=RIGHT'+'\n')
        for iy in range(vertNodes-1):
            g.write(str(nodeNumber[horizNodes-2,iy])+'\n')

        #g.write('*ELSET,ELSET=LOWPERMEABILITY\n')
        #for iy in range(vertNodes-1):
        #    for ix in range(horizNodes-1):
        #        coordinatesX=coordsX[ix,iy]+coordsX[ix+1,iy]+coordsX[ix,iy+1]+coordsX[ix+1,iy+1]		
        #        coordinatesY=coordsY[ix,iy]+coordsY[ix+1,iy]+coordsY[ix,iy+1]+coordsY[ix+1,iy+1]	
        #        if (coordinatesX<horizSize*0.1) or (coordinatesX>horizSize*0.9) or (coordinatesY<0.1*vertSizeMin):
        #            g.write(str(elemNumber[ix,iy])+'\n')
        #g.write('*ELSET,ELSET=HIGHPERMEABILITY\n')
        #for iy in range(vertNodes-1):
        #    for ix in range(horizNodes-1):
        #        coordinatesX=coordsX[ix,iy]+coordsX[ix+1,iy]+coordsX[ix,iy+1]+coordsX[ix+1,iy+1]
        #        coordinatesY=coordsY[ix,iy]+coordsY[ix+1,iy]+coordsY[ix,iy+1]+coordsY[ix+1,iy+1]
        #        if (coordinatesX>=horizSize*0.1) or (coordinatesX<=horizSize*0.9) or (coordinatesY>=0.1*vertSizeMin): 
        #            g.write(str(elemNumber[ix,iy])+'\n')        	
        g.close()


# change from here
def createRheologyFile(rheologyFileName):
    r = open(rheologyFileName, 'w')
    r.write('*MATERIAL,NAME=POROUSMEDIA'+'\n')
    r.write('*ELASTIC\n')
    r.write('1.e11,0.490\n')
    r.write('*EXPANSION \n')
    r.write('5.e-5\n')
    r.write('*PERMEABILITY,SPECIFIC=10000.\n')
    r.write(str(rechargePorosity)+',0.05\n')
    r.write(str(RHSPorosityDry)+',0.09\n')
    r.write(str(RHSPorosity)+',0.167\n')
    r.write(str(faultPorosity)+',0.5\n')
    r.write(str(shallowAquiferPorosity)+',1.0\n')
    #r.write('POROUS BULK MODULI\n')
    #r.write('1.E10\n')
    r.write('*CONDUCTIVITY\n')
    r.write('3.E-3\n')
    r.write('*DENSITY\n')
    r.write('3000\n')
    r.write('*DENSITY,PORE FLUID\n')
    r.write('1000\n')
    r.close()
# change until here

def createInputFile(inputFileName,geometryFileName):
    i = open(inputFileName, 'w')
    i.write('*HEADING'+'\n')
    i.write('input file for porous media flow'+'\n')
    i.write('*INCLUDE,INPUT='+geometryFileName+'\n')
    i.write('*SOLID SECTION, ELSET=BOX, MATERIAL=POROUSMEDIA'+'\n')
    i.write('*INCLUDE,INPUT='+rheologyFileName+'\n')
    i.write('*BOUNDARY, OP=NEW, TYPE=DISPLACEMENT'+'\n')
    i.write('LEFT, 1,,0.\n')
    i.write('RIGHT, 1,,0.\n')
    i.write('BOTTOM, 2,,0.\n')
    #i.write('LEFT, 8,,0.\n')
    #i.write('RIGHT, 8,,0.\n')
    #i.write('BOTTOM, 8,,0.\n')
    #i.write('TOP, 2,,0.\n')
    i.write('*INITIAL CONDITIONS,TYPE=SATURATION\n')
    i.write('NODES,1.\n')
    i.write('*INITIAL CONDITIONS,TYPE=RATIO\n')
    #i.write('NODES,1.\n')
    i.write('RECHARGEDOMAIN,0.05\n')
    i.write('RHSDRY,0.09\n')
    i.write('RHS,0.167\n')
    i.write('FAULTS,0.5\n')
    i.write('SHALLOWAQUIFER,1.0\n')
    i.write('*STEP\n')
    i.write('*GEOSTATIC\n')
    i.write('*DLOAD'+'\n')
    i.write('BOX, GRAV,' +str(gravity)+',0.,-1.,\n')
    i.write('*END STEP'+'\n')
    i.write('*STEP, NLGEOM,inc=10000'+'\n')
    i.write('*SOILS,UTOL=0.1,CREEP=NONE'+'\n')
    i.write(str(totalTime/10.)+','+str(totalTime)+','+str(totalTime/10000000.)+','+str(totalTime/10.)+'\n')
    i.write('*DLOAD'+'\n')
    i.write('BOX, GRAV,' +str(gravity)+',0.,-10.,\n')
    i.write('*DFLUX\n')
    i.write('LEFT,S4,0\n')
    i.write('RIGHT,S2,0\n')
    i.write('*Output, field, op=NEW, frequency=100'+'\n')
    i.write('*BOUNDARY'+'\n')
    for iy in range(1,vertNodes-1):
        # pressure
        i.write(str(nodeNumber[0,iy])+',8,,'+str((vertSizeMin-coordsY[0,iy])*10000.)+'\n')
        i.write(str(nodeNumber[horizNodes-1,iy])+',8,,'+str((vertSizeMax-coordsY[horizNodes-1,iy])*10000.)+'\n')
    for ix in range(0,horizNodes):
        i.write(str(nodeNumber[ix,0])+',8,,'+str((correctedVertSize[ix]-coordsY[ix,0])*10000.)+'\n')
        i.write(str(nodeNumber[ix,vertNodes-1])+',8,,'+str(0.)+'\n')
    i.write('*OUTPUT,FIELD,FREQ=1'+'\n')
    i.write('*Node Output,NSET=NODES'+'\n')
    i.write('U, COORD,POR,RVF'+'\n')
    i.write('*Element Output,ELSET=BOX'+'\n')
    i.write('E, ER, S, LE, CE, CEEQ,ENER,FV,POR,SAT,FLVEL,HFL,VOIDR,FLUVR,GELVR'+'\n')
    i.write('*OUTPUT, HISTORY, OP=NEW, frequency=1'+'\n')
    i.write('*ENERGY OUTPUT'+'\n')
    i.write('ALLAE, ALLCD, ALLFD, ALLIE, ALLKE, ALLPD, ALLSE, ALLVD, ALLWK, ETOTAL'+'\n')
    i.write('*END STEP'+'\n')

# HERE THE PROGRAM BEGINS
#create a separate file with the geometry
totalTime=3*9.467e+12 # 1Myrs
geometryFileName='geometry1.inp'
createGeometryFile(geometryFileName)
rheologyFileName='rheology.inp'
refName='kyle-hill-porosity-test'
inputFileName=refName+'.inp'
createRheologyFile(rheologyFileName)
createInputFile(inputFileName,geometryFileName)
# os.system('abaqus job='+refName+' \n')
print(refName + ' done')














kyle-hill-porosity-test done
